In [26]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import math
import time
import joblib
import os
from sklearn.svm import SVC
from calculator_module import calcFaceAngle, euclideanDistance, eye_aspect_ratio
from logistic_module import create_logistic_model
from whereWatching_module import create_AtanMean
import warnings
warnings.filterwarnings('ignore')

# 폰트 색상 지정
blue = (255, 0, 0)
green= (0, 255, 0)
red= (0, 0, 255)
white= (255, 255, 255) 

#from scipy.spatial import distance as dist
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# right eyes indices
RIGHT_EYE=[33, 160, 158, 133, 153, 144] 

# Left eyes indices 
LEFT_EYE =[263, 387, 385, 362, 380, 373] 

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)

angle_right, angle_left = create_AtanMean()

prevTime = time.time()
totalTime = 0
currTime = 0

# SVM모델 로드 or 생성
dir_path = './pkl/ear_logistic.pkl'
if os.path.isfile(dir_path):
    classifier = joblib.load(dir_path)
else :
    create_SVM_model()
    classifier = joblib.load(dir_path)
    
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                print("웹캠 인식 불가")
                continue

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # image로부터 랜드마크 반환
            results = face_mesh.process(image)
            
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # 반환된 랜드마크를 image에 덧씌움
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    mypose = face_landmarks
                    tmpList = []
                    for idx,lm in enumerate(mypose.landmark):
                        point=np.array([lm.x, lm.y, lm.z])
                        tmpList.append(point)
                
                ear = eye_aspect_ratio(image, tmpList, RIGHT_EYE, LEFT_EYE)
                
                blink = classifier.predict([[ear]])
                faceWatchingAngle = calcFaceAngle(tmpList)
              
                if blink:
                    t = time.time()
                    currTime = t - prevTime
                    prevTime = t
                    
                    totalTime += currTime

                    stime = totalTime.__str__()[0:2]+"sec"
                    cv2.putText(image, stime, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                    if totalTime > 4.0:
                        cv2.putText(image, "GET UP!!", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 3)
                        cv2.rectangle(image, (200,70), (210,80), blue, -1)
                        cv2.rectangle(image, (210,60), (220,80), blue, -1)
                        cv2.rectangle(image, (220,50), (230,80), blue, -1)
                        cv2.rectangle(image, (230,40), (240,80), blue, -1)
                    elif totalTime > 3.0:
                        cv2.rectangle(image, (200,70), (210,80), blue, -1)
                        cv2.rectangle(image, (210,60), (220,80), blue, -1)
                        cv2.rectangle(image, (220,50), (230,80), blue, -1)
                    elif totalTime > 2.0:
                        cv2.rectangle(image, (200,70), (210,80), blue, -1)
                        cv2.rectangle(image, (210,60), (220,80), blue, -1)
                    elif totalTime > 1.0:
                        cv2.rectangle(image, (200,70), (210,80), blue, -1)
                    
                else:
                    prevTime = time.time()
                    totalTime = 0.0
                
                if faceWatchingAngle>max(angle_left, angle_right) or faceWatchingAngle<min(angle_left, angle_right):
                    cv2.putText(image, "FOCUS!!!", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 3)
            else:
                cv2.putText(image, "SHOW FACE!!!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 3)
            
            cv2.imshow('video_lecture_monitoring', image)
            
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()